In [2]:
import importlib
import sys
from urllib.request import urlretrieve

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess()

# Let's do all of our imports here, too.
import numpy as np
import ipywidgets
import pydrake.all
import os
from IPython.display import display

from pydrake.all import RigidTransform, RotationMatrix, RollPitchYaw
import pydrake.multibody.jupyter_widgets
import pydrake.systems.jupyter_widgets

In /home/dani/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/dani/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/dani/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/dani/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/dani/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_c

In [9]:
def AddPlanarBinAndSimpleBox(plant, mass=0.1, mu=1.0, width=0.2, depth=0.05, height=0.3):
    # Parse bin model
    parser = pydrake.multibody.parsing.Parser(plant)
    bin_ = parser.AddModelFromFile("models/planar_bin_.sdf")
    
    # Weld bin to world
    plant.WeldFrames(
        plant.world_frame(),
        plant.GetFrameByName("bin_base", bin_),
        RigidTransform(RotationMatrix.MakeZRotation(np.pi/2.0), [0,0,-0.015])
    )

    planar_joint_frame = plant.AddFrame(
        pydrake.multibody.tree.FixedOffsetFrame("planar_joint_frame",
                                                plant.world_frame(),
                                                RigidTransform(RotationMatrix.MakeXRotation(np.pi/2))
                                               ))

    box_instance = plant.AddModelInstance("box")
    box_body = plant.AddRigidBody(
        "box_body",
        box_instance,
        pydrake.multibody.tree.SpatialInertia(mass=mass, p_PScm_E=np.array([0., 0., 0.]),
        G_SP_E=pydrake.multibody.tree.UnitInertia.SolidBox(width, depth, height))
    )
    
    # Set up collision
    if plant.geometry_source_is_registered():
        plant.RegisterCollisionGeometry(
            box_body,
            RigidTransform(),
            pydrake.geometry.Box(width-0.001, depth-0.001, height=0.001),
            "box_body", pydrake.multibody.plant.CoulombFriction(mu, mu)
        )
        i=0
        for x in [-width/2.0, width/2.0]:
            for y in [-depth/2.0, depth/2.0]:
                for z in [-height/2.0, height/2.0]:
                    plant.RegisterCollisionGeometry(
                        box_body,
                        RigidTransform([x, y, z]),
                        pydrake.geometry.Sphere(radius=1e-7), f"contact_sphere{i}",
                        pydrake.multibody.plant.CoulombFriction(mu, mu))
                    i += 1
        plant.RegisterVisualGeometry(
            box_body,
            RigidTransform(),
            pydrake.geometry.Box(width, depth, height),
            "box_body",
            [.5, .5, .9, 1.0])
    
    box_joint = plant.AddJoint(
        pydrake.multibody.tree.PlanarJoint("box_joint", planar_joint_frame, box_body.body_frame()))
    box_joint.set_default_translation([0, height/2.0])
    return box_instance

def AddPointFinger(plant):
    mu = 1.0
    finger = plant.AddModelInstance("finger")
    false_body1 = plant.AddRigidBody("false_body1", finger, pydrake.multibody.tree.SpatialInertia(0, [0,0,0], pydrake.multibody.tree.UnitInertia(0,0,0)))
    finger_body = plant.AddRigidBody("body", finger, pydrake.multibody.tree.SpatialInertia(mass=1.0, p_PScm_E=np.array([0., 0., 0.]),
            G_SP_E=pydrake.multibody.tree.UnitInertia(1.0, 1.0, 1.0)))
    shape = pydrake.geometry.Sphere(0.01)
    if plant.geometry_source_is_registered():
        plant.RegisterCollisionGeometry(finger_body, RigidTransform(), shape, "body", pydrake.multibody.plant.CoulombFriction(mu, mu))
        plant.RegisterVisualGeometry(finger_body, RigidTransform(), shape, "body", [.9, .5, .5, 1.0])
    finger_x = plant.AddJoint(pydrake.multibody.tree.PrismaticJoint("finger_x", plant.world_frame(), plant.GetFrameByName("false_body1"), [1, 0, 0], -.3, .3))
    plant.AddJointActuator("finger_x", finger_x)
    finger_z = plant.AddJoint(pydrake.multibody.tree.PrismaticJoint("finger_z", plant.GetFrameByName("false_body1"), plant.GetFrameByName("body"), [0, 0, 1], 0.0, 0.5))
    finger_z.set_default_translation(0.25)
    plant.AddJointActuator("finger_z", finger_z)

    return finger

class PointFingerForceControl(pydrake.systems.framework.LeafSystem):

    def __init__(self, plant):
        pydrake.systems.framework.LeafSystem.__init__(self)
        self._plant = plant

        self.DeclareVectorInputPort(
            "desired_contact_force", pydrake.systems.framework.BasicVector(2))
        self.DeclareVectorOutputPort(
            "finger_actuation", pydrake.systems.framework.BasicVector(2), 
                                    self.CalcOutput)

    def CalcOutput(self, context, output):
        finger_mass = 1
        g = self._plant.gravity_field().gravity_vector()[[0,2]]

        desired_force = self.get_input_port(0).Eval(context)
        output.SetFromVector(-finger_mass*g - desired_force)

In [11]:
from ipywidgets import FloatSlider, Layout

def force_control_point_finger(teleop=True):
    builder = pydrake.systems.framework.DiagramBuilder()

    plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    box = AddPlanarBinAndSimpleBox(plant)
    finger = AddPointFinger(plant)
    plant.GetJointByName("finger_x").set_default_translation(0.15)
    plant.GetJointByName("finger_z").set_default_translation(0.025)
    plant.Finalize()

    vis = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder, scene_graph, delete_prefix_on_load=False)
    vis.vis['drake']['4'].delete()  # Only flush the gripper geometry.
    #vis.set_planar_viewpoint(xmin=-.5, xmax=.5, ymin=-0.1, ymax=0.6)
    contact_visualizer = builder.AddSystem(pydrake.systems.meshcat_visualizer.MeshcatContactVisualizer(vis, plant=plant, contact_force_radius=0.005, contact_force_scale=40.0))
    builder.Connect(plant.get_contact_results_output_port(),
                    contact_visualizer.GetInputPort("contact_results"))

    class RotateBoxOnTopCorner(pydrake.systems.framework.LeafSystem):

        def __init__(self, plant):
            pydrake.systems.framework.LeafSystem.__init__(self)
            self._plant = plant

            # One state variable for the integral error.
            self.DeclareContinuousState(1)

            self.DeclareVectorInputPort("desired_box_angle",
                pydrake.systems.framework.BasicVector(1))
            self.DeclareVectorInputPort(
                "box_state", pydrake.systems.framework.BasicVector(6))
            self.DeclareVectorInputPort(
                "finger_state", pydrake.systems.framework.BasicVector(4))
            self.DeclareVectorOutputPort(
                "finger_force", pydrake.systems.framework.BasicVector(2), 
                                        self.CalcOutput)

        def DoCalcTimeDerivatives(self, context, derivatives):
            # Compute integral of the angle error.
            theta_desired = self.get_input_port(0).Eval(context)[0]
            theta = self.get_input_port(1).Eval(context)[2]
            derivatives.get_mutable_vector().SetFromVector([theta_desired - theta])

        def CalcOutput(self, context, output):
            box_mass = 0.411  # Let's revisit whether we need this!
            mu_A = 1.0
            mu_C = 1.0
            g = -self._plant.gravity_field().gravity_vector()[2]

            theta_desired = self.get_input_port(0).Eval(context)[0]
            box_state = self.get_input_port(1).Eval(context)
            theta = box_state[2]
            theta_dot = box_state[5]

            R_WC = np.array([[-np.sin(theta), -np.cos(theta)], [np.cos(theta), -np.sin(theta)]])
            R_CW = R_WC.T
            finger_state = self.get_input_port(2).Eval(context)

            prog = pydrake.solvers.mathematicalprogram.MathematicalProgram()
            f_C_W = prog.NewContinuousVariables(2, "f_C_W")  # force from finger to box
            f_C_C = np.matmul(R_CW, f_C_W)

            # PID control on f_CC[1]
            kp = 100.0
            ki = 1.0
            kd = 2.0*np.sqrt(kp)  # should really be kp*I on inside
            integral_error = context.get_continuous_state_vector().GetAtIndex(0)
            pid = kp*(theta_desired - theta) + ki*integral_error - kd*theta_dot
            prog.AddCost((f_C_C[0] - pid)**2)

            prog.AddConstraint(f_C_W[0] <= mu_A * (box_mass*g - f_C_W[1]))
            prog.AddConstraint(-f_C_W[0] <= mu_A * (box_mass*g - f_C_W[1]))
            prog.AddConstraint(f_C_C[1] >= 0.1)  # To ensure I make and stay in contact.
            prog.AddConstraint(f_C_C[0] <= mu_C * f_C_C[1])
            prog.AddConstraint(-f_C_C[0] <= mu_C * f_C_C[1])

            result = pydrake.solvers.mathematicalprogram.Solve(prog)
            
            if not result.is_success():
                print(result.GetInfeasibleConstraintNames(prog))
                output.SetFromVector([0, 0])
                return
            
            f_C_W = result.GetSolution(f_C_W)
            output.SetFromVector(-f_C_W)


    controller = builder.AddSystem(PointFingerForceControl(plant))
    builder.Connect(controller.get_output_port(), plant.get_actuation_input_port())

    rotate_box = builder.AddSystem(RotateBoxOnTopCorner(plant))
    builder.Connect(rotate_box.get_output_port(), controller.get_input_port())
    builder.Connect(plant.get_state_output_port(box), rotate_box.get_input_port(1))
    builder.Connect(plant.get_state_output_port(finger), rotate_box.get_input_port(2))

    if teleop:
        slider = FloatSlider(value=0, min=0, max=np.pi/2, step=0.05, continuous_update=True, description="desired_box_angle", layout=Layout(width="'100'"))
        teleop = builder.AddSystem(pydrake.systems.jupyter_widgets.WidgetSystem([slider]))
        builder.Connect(teleop.get_output_port(), rotate_box.get_input_port(0)) 
    else:
        theta_trajectory = pydrake.trajectories.PiecewisePolynomial.FirstOrderHold(
            [0, 2, 20], [[0, 0, np.pi/2.0]])
        trajectory_source = builder.AddSystem(pydrake.systems.primitives.TrajectorySource(theta_trajectory))
        builder.Connect(trajectory_source.get_output_port(), rotate_box.get_input_port(0))

    diagram = builder.Build()
    simulator = pydrake.systems.analysis.Simulator(diagram)

    simulator.set_target_realtime_rate(.5)  # slow motion!
    vis.start_recording()
    simulator.AdvanceTo(theta_trajectory.end_time())
    vis.stop_recording()
    vis.vis['drake/contact_forces'].delete()  # TODO: make my recording include the contact forces!
    vis.publish_recording()

force_control_point_finger(teleop=False)

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.
